# PIM_DATA_WITH_ATTB
   - Module name : PIM_DATA_WITH_ATTB.ipynb 
   - purpose: 
      Based on the value 'NAM_IN_KEY_LNG_NAM' to extract 'ITM_ID', 'ATB_CD', 'ATB_VLU' from the table 'CLS_SPC_ATB'  
        please use this file 'PIM_Flavor_Attributes_current.csv',  to list out the unique flavors available for each department
   - input:
       - sheet_name 'Flavor(s)' = in Flavor_CLS_SPC_ATB .xlsx
       - sheet_name 'Other Flavor' = in Flavor_CLS_SPC_ATB .xlsx   
       - ITM_ID_GTIN_Map_100323.csv  i  the 
       - PIM_Data_New_50_82Mn.csv  (5082212, 24)
   - Output:  PIM_DATA_WITH_FLAVOR_ATTB_yymmdd.xlsxF
   - Written by: Sophia Yue
   - Date: 10-08-23

In [1]:
def f_prepare_pim_data():
    
    keep_col =['GTIN', 'PMY_DPT_CD', 'PMY_DPT_DSC',
    'REC_DPT_CD', 'REC_DPT_DSC', 'DPT_CD', 'DPT_DSC', 'COM_CD', 'COM_DSC',
    'SUBCOM_CD', 'SUBCOM_DSC','VND_ECOM_DSC']    
    df_pim_data_new = pd.read_csv(path + 'PIM_Data_New_50_82Mn.csv',  dtype={"GTIN": 'int64', "ITM_ID": 'int64'},  usecols= keep_col )
    df_pim_data_new.rename(columns ={'GTIN': 'GTIN_NO'}, inplace = True) 
    
    df_pim_data_new_parquet  = pd.read_parquet(path + 'ITM_DSC_ATTRIBUTES_NEW_parquet.gzip', columns=['GTIN_NO', 'GTIN_NAM', 'ITM_LVL_DSC'] )
    df_pim_data=  pd.merge(df_pim_data_new, df_pim_data_new_parquet,  on ='GTIN_NO', how = 'left')             
     
    # NW df_gtin_map  = pd.read_csv(path + 'IIM_ID_GTIN_Map_101123.csv',  dtype={"GTIN_NO": 'int64', "ITM_ID": 'int64'})

    df_gtin_map  = f_read(path, 'ITM_ID_GTIN_Map_102323.csv')
    df_gtin_map['GTIN_NO'] = df_gtin_map['GTIN_NO'].astype(np.float64).astype(np.int64)
    df_gtin_map['ITM_ID']  = df_gtin_map['ITM_ID'].astype(np.float64).astype(np.int64)    
    
    df_pim_data_adj = pd.merge( df_pim_data, df_gtin_map, on=['GTIN_NO'],  how='inner')
    print (f' df_pim_data_adj.shape { df_pim_data_adj.shape} \n columns: {df_pim_data_adj.columns.values}')
               
    print (f'df_pim_data_adj.dtypes: \n {df_pim_data_adj.dtypes}')
    print (f'head df_pim_data_adj: \n {df_pim_data_adj.head()}')
    return  df_pim_data_adj, df_pim_data_new,  df_pim_data, df_pim_data_new_parquet, df_gtin_map  

In [2]:
def f_comb_atb_gtin(df_atb, df_gtin, col_order, save_file, save_loc):
    df_mrg = pd.merge(df_atb, df_gtin, on =  'ITM_ID', how ='left')
    df_mrg = df_mrg[col_order]
    file_name = save_file.split('.')[0]
    file_ext = save_file.split('.')[1]
    dte = date.today().strftime('%m%d%y')
    path_file = save_loc + file_name + '_' +  dte + '.' + file_ext
    print(f'path_file = {path_file}')
    if file_ext   == 'csv':   df_mrg.to_csv(path_file,  index=False )  
    elif file_ext == 'xlsx':  df_mrg.to_excel(path_file,  index=False ) 
    else: pass
    print(f'df_mrg.shape {df_mrg.shape} \n columns: {df_mrg.columns}')
    print (f'dtypes: \n {df_mrg.dtypes}')
    print(f'df_mrg.head() \n {df_mrg.head()}' )
    return df_mrg

In [3]:
#df_scent_gtin = f_comb_atb_gtin(df_scent, df_pim_data_adj,scent_gtin_col, scent_file, path_atb)

In [4]:
import pandas as pd, numpy as np
from datetime import date
pd.options.display.float_format = '{:20,.2f}'.format 
pd.set_option("display.max_rows", None)
path = 'C:\\users\\iny2819\\Kroger\\Data\\' 
path_atb = path + 'ATB\\'
gtin_col =['ITM_ID', 'GTIN_NO', 'GTIN_NAM', 'PMY_DPT_CD', 'PMY_DPT_DSC',
       'REC_DPT_CD', 'REC_DPT_DSC', 'DPT_CD', 'DPT_DSC', 'COM_CD', 'COM_DSC',
       'SUBCOM_CD', 'SUBCOM_DSC', 'ITM_LVL_DSC', 'VND_ECOM_DSC']
    

In [5]:
path_code = 'C:\\users\\iny2819\\kroger\\Code\\'  
f_com_code = path_code + "com_code.py"
exec(compile(open(f_com_code , "rb").read(), f_com_code, 'exec' ))

In [6]:
df_pim_data_adj, df_pim_data_new,  df_pim_data, df_pim_data_new_parquet, df_gtin_map   = f_prepare_pim_data() 

file ITM_ID_GTIN_Map_102323.csv shape (5213311, 2) 
 columns: Index(['GTIN_NO', 'ITM_ID'], dtype='object')
dtypes: 
 GTIN_NO    object
ITM_ID     object
dtype: object
head: 
           GTIN_NO     ITM_ID
0  00202442400008  800084981
1  00011110001207  800084982
2  10011110001204  800084983
3  00810845012096  800084984
4  10810845012093  800084985
 df_pim_data_adj.shape (5081808, 15) 
 columns: ['GTIN_NO' 'PMY_DPT_CD' 'PMY_DPT_DSC' 'REC_DPT_CD' 'REC_DPT_DSC' 'DPT_CD'
 'DPT_DSC' 'COM_CD' 'COM_DSC' 'SUBCOM_CD' 'SUBCOM_DSC' 'VND_ECOM_DSC'
 'GTIN_NAM' 'ITM_LVL_DSC' 'ITM_ID']
df_pim_data_adj.dtypes: 
 GTIN_NO           int64
PMY_DPT_CD      float64
PMY_DPT_DSC      object
REC_DPT_CD      float64
REC_DPT_DSC      object
DPT_CD          float64
DPT_DSC          object
COM_CD          float64
COM_DSC          object
SUBCOM_CD       float64
SUBCOM_DSC       object
VND_ECOM_DSC     object
GTIN_NAM         object
ITM_LVL_DSC      object
ITM_ID            int64
dtype: object
head df_pim_data_adj: 


In [10]:
df_flavor_all = f_read(path_atb, 'Flavor_CLS_SPC_ATB_102323.xlsx')
flavor_gtin_col = gtin_col + [ 'Flavor_Code', 'Flavor_Value', 'comb_flavor_cod_val']
flavor_file = 'PIM_DATA_WITH_FLAVOR_ATTB.xlsx'
df_flavor_gtin = f_comb_atb_gtin(df_flavor_all, df_pim_data_adj,flavor_gtin_col, flavor_file, path_atb)

file Flavor_CLS_SPC_ATB_102323.xlsx shape (60340, 5) 
 columns: Index(['NAM_IN_KEY_LNG_NAM', 'ITM_ID', 'Flavor_Code', 'Flavor_Value',
       'comb_flavor_cod_val'],
      dtype='object')
dtypes: 
 NAM_IN_KEY_LNG_NAM     object
ITM_ID                  int64
Flavor_Code            object
Flavor_Value           object
comb_flavor_cod_val    object
dtype: object
head: 
   NAM_IN_KEY_LNG_NAM     ITM_ID    Flavor_Code Flavor_Value  \
0          Flavor(s)  800084984           NONE         NONE   
1          Flavor(s)  800085035           MINT         MINT   
2          Flavor(s)  800085038  MISCELLANEOUS          NaN   
3          Flavor(s)  800085038         COFFEE       COFFEE   
4          Flavor(s)  800085047  MISCELLANEOUS          NaN   

  comb_flavor_cod_val  
0                NONE  
1                MINT  
2       MISCELLANEOUS  
3              COFFEE  
4       MISCELLANEOUS  
path_file = C:\users\iny2819\Kroger\Data\ATB\PIM_DATA_WITH_FLAVOR_ATTB_102423.xlsx
df_mrg.shape (60340, 18) 

In [8]:
Stop

NameError: name 'Stop' is not defined

In [ ]:
df_scent a= f_read(path_atb, 'Scent_CLS_SPC_ATB.xlsx')
scent_gtin_col = gtin_col + [ 'Scent_Code', 'Scent_Value']
df_scent = f_read(path_atb, 'Scent_CLS_SPC_ATB.xlsx')
scent_file = 'PIM_DATA_WITH_SCENT_ATTB.xlsx'
df_scent_gtin = f_comb_atb_gtin(df_scent, df_pim_data_adj,scent_gtin_col, scent_file, path_atb)

In [ ]:

df_scent = f_read(path_atb, 'Scent_CLS_SPC_ATB.xlsx')
df_scent_gtin = pd.merge(df_scent, df_pim_data_adj, on = 'ITM_ID', how ='left')                  

In [ ]:
#df_secnt_gtin = pd.merge(df_scent, df_pim_data_adj, on =  'ITM_ID', how ='left') 
df_scent_gtin.columns 

In [ ]:
keep_col =['ITM_ID', 'GTIN_NO', 'GTIN_NAM', 'PMY_DPT_CD', 'PMY_DPT_DSC',
       'REC_DPT_CD', 'REC_DPT_DSC', 'DPT_CD', 'DPT_DSC', 'COM_CD', 'COM_DSC',
       'SUBCOM_CD', 'SUBCOM_DSC', 'ITM_LVL_DSC', 'VND_ECOM_DSC',
       'Scent_Value', 'Scent_Code']
#df_pim_data_parquet = df_pim_data_new_parquet[df_pim_data_new_parquet.columns[keep_col]]  
df_scent_gtin= df_scent_gtin[keep_col] 

In [ ]:
df_scent_gtin_col = ['ITM_ID', 'Scent_Code', 'Scent_Value', 'GTIN_NO', 'PMY_DPT_CD',
       'PMY_DPT_DSC', 'REC_DPT_CD', 'REC_DPT_DSC', 'DPT_CD', 'DPT_DSC',
       'COM_CD', 'COM_DSC', 'SUBCOM_CD', 'SUBCOM_DSC', 'VND_ECOM_DSC',
       'GTIN_NAM', 'ITM_LVL_DSC'],
df_secnt_gtin = 

In [ ]:
 df_pim_data_adj.dtypes

In [ ]:
 df_pim_data_adjx = pd.merge( df_pim_data, df_gtin_map, on=['GTIN_NO'],  how='left') 
 df_pim_data_adjx.dtypes

In [ ]:
 df_pim_data_adjx.head()

In [ ]:
 df_flavor_gtin.isnull().sum()

In [ ]:
 df_pim_data_adj.dtypes

In [ ]:
 df_flavor_gtin['GTIN_NO'] = df_flavor_gtin['GTIN_NO'].apply( lambda x: int(x) if x != np.NaN else 0)

In [ ]:
df_gtin_map  = pd.read_csv(path + 'IIM_ID_GTIN_Map_101123.csv',  dtype={"GTIN_NO": 'int64', "ITM_ID": 'int64'})

In [ ]:
df_gtin_map  = f_read(path, 'ITM_ID_GTIN_Map_101123_x.csv')

In [ ]:
df_gtin_map  = f_read(path, 'ITM_ID_GTIN_Map_102323.csv')

In [ ]:
df_gtin_map  = pd.read_csv(path + 'IIM_ID_GTIN_Map_102323.csv',  dtype={"GTIN_NO": 'int64', "ITM_ID": 'int64'})